In [92]:
import numpy as np
import math
from collections import Counter
from time import sleep

NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4
MINMAX_DEPTH = 3
MONTECARLO_SAMPLES = 3
MONTECARLO_STEPS = 10
EVAL_MODE = 1 #0 for minmax + montecarlo, 1 for montecarlo only

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"


Basic Functions

In [93]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

Encode/decode a board

MinMax

In [94]:



def minmax(board, player, depth, a,b):
    if(four_in_a_row(board,player)):
        return (-1,player, depth)
    
    if(depth>= MINMAX_DEPTH):
        #score, _, __, ___ = mc_simulate(board,player)
        return (-1,montecarlo_wrap(board, player), depth)

    evaluations = list()
    #print(valid_moves(board))
    cur_a = -math.inf
    cur_b = math.inf
    for m in valid_moves(board):
        play(board, m, player)
        val = minmax(board,-player, depth+1, cur_a, cur_b)
        evaluations.append((m,val[1], val[2]))
        take_back(board, m)
        if alpha_beta_stop(a,b,player, val[1]):
            #print("cut!")
            break
        cur_a, cur_b=alpha_beta_update(cur_a,cur_b, player, val[1])
    
    #print(f"depth: {depth} player: {player} ab: {a} {b} cur_ab: {cur_a} {cur_b} evals: {evaluations} ")
    if player>0:
        return max(evaluations, key = lambda k: k[1]*100 -k[2] )
    else:
        return min(evaluations, key=lambda k: k[1]*100 + k[2])

def alpha_beta_update(a,b, player, cur):
    if player <0 and cur < b:
        #print("update")
        return a, cur
    elif player > 0 and cur> a:
        #print("update"); 
        return cur, b 
    return a,b

def alpha_beta_stop(a, b, player, cur):
    if player <0 and cur <= a:
        return True
    elif player > 0 and cur >= b:
        return True 
    return False





Montecarlo

In [95]:
class node:
    def __init__(self, board, num, den, player, parent):
        self.board = np.copy(board)
        self.num = num 
        self.den = den
        self.player=player
        self.parent=parent
        self.children=list()
        self.terminal = four_in_a_row(board, -player)
        self.n_children = 0
        self.terminated_childrend = 0
    
    def simulate(self):
        
        #print(f"simulating")
        #print(self.board)
        #get winrate statisticsf four_in_a_row(board, -p):
    
        
        _,draws,winmax,winmin = mc_simulate(self.board, self.player)

        if(self.terminal):
            draws = 0
            winmax=MONTECARLO_SAMPLES
            winmin=MONTECARLO_SAMPLES
        else:
            _,draws,winmax,winmin = mc_simulate(self.board, self.player)
        
        #derive numerator and denominator change
        if self.player > 0:
            self.num += winmax + 0.5* draws
        else:
            self.num += winmin + 0.5* draws
        
        self.den += MONTECARLO_SAMPLES
        #print(f"winmax {winmax} winmin {winmin} draws {draws}")
        #backpropagate evaluation
        if self.parent:
            self.parent.backprop(winmax,winmin, draws, MONTECARLO_SAMPLES)
            
        

    def backprop(self,winmax,winmin,draws, den):
        #print("bp")
        if self.player > 0:
            self.num += winmax + 0.5* draws
        else:
            self.num += winmin + 0.5* draws
        
        self.den += den
        if self.parent:
            self.parent.backprop(winmax,winmin, draws, den)
    
    def expand(self):
        #print("expanding")
        #print(self.board)
        
        for m in valid_moves(self.board):
            
            play(self.board,m,self.player)
            c = node(self.board,0,0,-self.player, self)
            c.simulate()
            self.children.append(c)
            
            take_back(self.board, m)
            if(c.terminal and self.parent):
                #the current move gives the opponent a win in 1 move, don't waste time in this branch
                self.parent.num = -math.inf
                break
        
        #print(f"winrate for this {self.winrate()}")
        return self.children.copy()
    def winrate(self):
        #simplified UCT
        return self.num/self.den
         

def mc_select(node):
    if not node.children:
        return node
    if not list(filter(lambda c: c.terminal == False, node.children)):
        return node
    cur_best = max(filter(lambda c: c.terminal == False, node.children), key= lambda n: n.winrate())

    return mc_select(cur_best)

def _mc(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0



def mc_simulate(board, player):
    
    cnt = Counter(_mc(np.copy(board), player) for _ in range(MONTECARLO_SAMPLES))
    return (cnt[1] - cnt[-1]) / MONTECARLO_SAMPLES, cnt[0], cnt[1], cnt[-1]

def montecarlo_wrap(board, player):
    #define root node
    root = node(board, 0,0, player, None)
    if(root.terminal):
        return 1.0
    root.expand()

    #main loop
    for i in range(MONTECARLO_STEPS):
        #selection: get leaf by going through path with best winrates 
        next_node = mc_select(root)

        

        #expand the node (expands + simulation + backpropagation)
        children = next_node.expand()

    #return winrate of root node
    return root.winrate() 



Evaluation wrapper

In [96]:
def montecarlo_only(board, player):
    #montecarlo evaluation for each possible move, then choose best move
    evaluations = list()
    for m in valid_moves(board):
        play(board, m, player)
        winrate = montecarlo_wrap(board, player)
        evaluations.append((m,winrate))
        take_back(board, m)
        #print(f"{m} gives {winrate} winrate")
    
    return max(evaluations, key = lambda k: k[1])
        

#returns: move, score. Score is in range (-1,1) if using minmax+montecarlo (EVAL_MODE = 0), a winrate in range (0,1) with montecarlo only 
def eval_board(board, player):
    if four_in_a_row(board, 1):
        # Alice won
        return -1,1
    elif four_in_a_row(board, -1):
        # Bob won
        return -1,1
    else:
        # Not terminal, let's simulate...
        if EVAL_MODE ==0:
            
            eval= minmax(board, player,0, -math.inf, math.inf)
            return eval[0], eval[1]
        elif EVAL_MODE == 1:
            eval = montecarlo_only(board, player)
            return eval[0], eval[1]
        else:
            print("wrong evaluation mode selected, check the EVAL_MODE constant")
            return -1,0


Example

In [97]:


board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
play(board, 3, 1)
play(board, 0, -1)
play(board, 4, 1)
play(board, 0, -1)
#play(board, 5, 1)
#play(board, 2, -1)
play(board, 0, -1)
#play(board,2,-1)
print(board)
#eval_board(board, -1)
#play(board, mv[0], )



[[-1 -1 -1  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]


Simple Human vs AI handler (run to play)

In [98]:
def match(board):
    start= input("who will start? ('h' for human, 'a' for AI, default is human): ")
    ai=-1
    human = 1
    if(start == 'a'):
        human = -1
        ai = -1
    curplayer = 1

    while valid_moves(board):
        sleep(0.5)
        print(f"legal moves are {valid_moves(board)}")
        if curplayer == human:
            
            move = -1
            
            while not False:
                move = int(input("type your move:"))
                if move not in valid_moves(board):
                    print("illegal move!")
                else:
                    break
            play(board, move, curplayer)
            print(board)
        elif curplayer == ai:
            move = eval_board(board, curplayer)
            print(f"ai played {move[0]}")
            play(board,move[0], curplayer)
            print(board)
        
        if four_in_a_row(board, human):
            print(board)
            print("you won!")
            break
        elif four_in_a_row(board, ai):
            print(board)
            print("ai won!!!")
            break
        
        curplayer*=-1
        

board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
match(board)

legal moves are [0, 1, 2, 3, 4, 5, 6]
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
legal moves are [0, 1, 2, 3, 4, 5, 6]
ai played 3
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
legal moves are [0, 1, 2, 3, 4, 5, 6]
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
legal moves are [0, 1, 2, 3, 4, 5, 6]
ai played 2
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
legal moves are [0, 1, 2, 3, 4, 5, 6]
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
legal moves are [0, 1, 2, 3, 4, 5, 6]
ai played 6
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0 